### Introduction
Due to the high rate at which the Dutch society is aging, the homecare sector is faced with two major challenges:
1. An increasing demand for home care services
2. A diminishing pool of home care professionals

Home care organizations need to find ways to do more with less. Some solutions will lie in the technological domain. For instance automating the most time-consuming processes will allow for shifting more resources towards actual care-giving. One process that can be automated is planning.

More text .....

### Setup necessary packages

In [17]:
import ipyleaflet as ipl
import seaborn as sns
from time import sleep
import ipywidgets as widgets
import networkx
import carinova_data as cd
import instance
import gomea
import pandas as pd
import osmnx as ox
ox.config(use_cache=True, log_console=True)


2022-03-21 14:38:46 Configured OSMnx 1.1.2
2022-03-21 14:38:46 HTTP response caching is on


### Inspect input data
The input data comes from an Excel file. Each row represents one care activity.

In [4]:
url = "example_data.xlsx"
source_df = pd.read_excel(url)
source_df.head()

,activity_id,client_nr,shift_code,shift_id,duration,tw_start,tw_end,tw_bool,activity_level_name,activity_level,latitude,longitude
0,1,2000140,AVP1-2233,0,5,NaN,NaN,0,PV 2+,2,52.269695,6.164124
1,2,705026,AVP1-2233,0,5,NaN,NaN,0,PV 3,3,52.267812,6.175048
2,3,705026,AVP1-2233,0,10,NaN,NaN,0,PV 2+,2,52.266598,6.168108
3,4,811647,AVP1-2233,0,10,NaN,NaN,0,PV 2+,2,52.269764,6.176036
4,5,819573,AVP1-2233,0,15,NaN,NaN,0,PV 2+,2,52.267690,6.166916


### Read in data from Excel file

The `carinova_data.py` file contains functions for extracting and transforming the data. All the paths are hard coded in the `fetch_data()` function. The final output is a dictionary.

In [5]:
input_data = cd.fetch_data('city')
keys = input_data.keys()
items = input_data.items()
values = input_data.values()
print(keys)

dict_keys(['n', 'v', 'p', 'd', 'tw', 'Q', 'u', 'ss', 'route', 'arrival', 'arrival_nobase', 'score', 'dist', 'wt', 'ot'])


### Build Instance
The function containing the genetic algorithm needs an Instance `class`. This `class` takes in the first 8 variables of the dictionary as input.

In [6]:
n, v, p, d, tw, Q, u, ss = list(values)[:8]
ins = instance.Instance(n=n, v=v, p=p, d=d, tw=tw, Q=Q, u=u, ss=ss)
ins.__dict__.keys()

dict_keys(['n', 'v', 'd', 'p', 'tw', 'Q', 'u', 'ss', 'feasibleShiftsForClients'])

### Solve for optimal routes
The function` gomea_solve()` contains the genetic algorithm. It requires an Instance object and returns a dictionary.

In [7]:
results = gomea.gomea_solve(ins)
results.keys()

evolution cycle 0 finished in 0:00:17.670588
evolution cycle 1 finished in 0:00:18.000110
evolution cycle 2 finished in 0:00:19.169879
evolution cycle 3 finished in 0:00:19.663607
evolution cycle 4 finished in 0:00:18.991377
evolution cycle 5 finished in 0:00:18.936459
evolution cycle 6 finished in 0:00:18.806922
evolution cycle 7 finished in 0:00:18.475936
evolution cycle 8 finished in 0:00:18.285461
evolution cycle 9 finished in 0:00:19.562298
evolution cycle 10 finished in 0:00:20.021452
evolution cycle 11 finished in 0:00:18.694343
evolution cycle 12 finished in 0:00:19.445023
evolution cycle 13 finished in 0:00:22.718267
evolution cycle 14 finished in 0:00:18.744917
evolution cycle 15 finished in 0:00:18.400974
evolution cycle 16 finished in 0:00:18.511982
evolution cycle 17 finished in 0:00:19.431806
evolution cycle 18 finished in 0:00:18.273945
evolution cycle 19 finished in 0:00:18.334401


total elapsed time: 0:06:20.139747


dict_keys(['params', 'gen_count', 'time_track', 'route', 'arrival', 'score', 'distance', 'waiting_time', 'shift_overtime', 'progress', 'pop_means', 'instance'])

### Data dictionary
Before actually drawing the routes all the necessary data needs to be combined in a logical data structure. As the selected variables have different lengths (single values, lists) a suitable data type is a dictionary. The following functions takes in the source data, the output of the genetic algorithm, a street network and a the starting coordinatess (latitude and longitude). It returns a dictionary containing all the data for building the maps and routes.

In [3]:
# Function combining data from source, gomea and street network into dictionary
def create_dict(source_df, gomea_dic, graph_url, base_location):
    # Initialize
    selection = ['activity_id', 'latitude', 'longitude']
    source_data = source_df[selection]
    routes_dic = {}
    start = base_location
    graph = ox.load_graphml(graph_url)
    gdf_nodes = ox.graph_to_gdfs(graph)[0]
    
    # For every route in gomea_dic create an entry in routes_dic with a route_id
    for rx, route in enumerate(gomea_dic['route']):
        location_dic = {}
        # For every location in the route create an entry with a location_id, a start location and a path list from start to location
        for lx, activity in enumerate(route):
            activity_id, latitude, longitude = list(source_data[source_data['activity_id'] == activity].iloc[0])
            location_dic[lx] = {'route_id': rx,
                                'activity_id': activity_id,
                                'start': start,
                                'location': [latitude, longitude]
                                }
            y1, x1 = start
            y2, x2 = location_dic[lx]['location']
            nodes = ox.nearest_nodes(G=graph, X=[x1, x2], Y=[y1, y2])
            path_nodes = networkx.shortest_path(graph, nodes[0], nodes[1])
            path_coord = gdf_nodes.loc[path_nodes][['x', 'y']]
            path = []
            for point in path_coord.values:
                path.append([point[1], point[0]])
                location_dic[lx]['path'] = path
                
            # Set new start location
            start = location_dic[lx]['location']
            
        routes_dic[rx] = location_dic
    return(routes_dic)
            
            

In [21]:
routes_dict = create_dict(source_df=source_df, gomea_dic=results, graph_url="deventer_graph.graphml",
            base_location=[52.27, 6.17])
routes_dict[0]

2022-03-21 15:15:50 Converting node, edge, and graph-level attribute data types
2022-03-21 15:15:50 Loaded graph with 20870 nodes and 39886 edges from "deventer_graph.graphml"
2022-03-21 15:15:51 Created nodes GeoDataFrame from graph
2022-03-21 15:15:55 Created edges GeoDataFrame from graph
2022-03-21 15:15:55 Created nodes GeoDataFrame from graph
2022-03-21 15:15:55 Created nodes GeoDataFrame from graph
2022-03-21 15:15:55 Created nodes GeoDataFrame from graph
2022-03-21 15:15:55 Created nodes GeoDataFrame from graph
2022-03-21 15:15:55 Created nodes GeoDataFrame from graph
2022-03-21 15:15:55 Created nodes GeoDataFrame from graph
2022-03-21 15:15:55 Created nodes GeoDataFrame from graph
2022-03-21 15:15:55 Created nodes GeoDataFrame from graph
2022-03-21 15:15:56 Created nodes GeoDataFrame from graph
2022-03-21 15:15:56 Created nodes GeoDataFrame from graph
2022-03-21 15:15:56 Created nodes GeoDataFrame from graph
2022-03-21 15:15:56 Created nodes GeoDataFrame from graph
2022-03-21 1

{0: {'route_id': 0,
  'activity_id': 59.0,
  'start': [52.27, 6.17],
  'location': [52.269551873372464, 6.176467448318099],
  'path': [[52.2698226, 6.1705944],
   [52.270096, 6.1714894],
   [52.2700804, 6.1715016],
   [52.2697, 6.1718],
   [52.26983, 6.17218],
   [52.26995, 6.17256],
   [52.2701945, 6.1733059],
   [52.2702354, 6.1734142],
   [52.2696162, 6.1739567],
   [52.2693092, 6.1742035],
   [52.2690495, 6.1744278],
   [52.2686323, 6.1747899],
   [52.2685221, 6.1748921],
   [52.2683988, 6.1749931],
   [52.26832, 6.17506],
   [52.2683496, 6.1751902],
   [52.2683778, 6.1753957],
   [52.2683996, 6.1756129],
   [52.268397, 6.1760867],
   [52.2683947, 6.1762058],
   [52.2683913, 6.1764228],
   [52.2684873, 6.1764242],
   [52.2686626, 6.1764269],
   [52.2687178, 6.176425],
   [52.2689551, 6.1764453],
   [52.2695902, 6.1765047]]},
 1: {'route_id': 0,
  'activity_id': 2.0,
  'start': [52.269551873372464, 6.176467448318099],
  'location': [52.26781209712188, 6.1750484228651565],
  'path': 

### Visualize routes
The quality of the calculated routes can be assessed by plotting them on a map. Each client can be plotted as a point and given a color depending on the route_id.

In [26]:
n_colors = len(routes_dict)
colors = sns.color_palette("Set2", n_colors).as_hex()

route_id = 3

route_dict_flr = routes_dict[route_id]

# Calculate map center and set bounds
latitudes = pd.Series(dtype="float64")
longitudes = pd.Series(dtype="float64")
for point in route_dict_flr.values():
    lat, lon = pd.Series([point["location"][0]]), pd.Series([point["location"][1]])
    latitudes = pd.concat([latitudes, lat])
    longitudes = pd.concat([longitudes, lon])
center = [latitudes.mean(), longitudes.mean()]
sw = [latitudes.min(), longitudes.min()]
ne = [latitudes.max(), longitudes.max()]
m1 = ipl.Map(center=center)
m1.fit_bounds([sw, ne])
m1.layout.height = '800px'

# Build route layer
trace = []
for point in route_dict_flr.values():
    for path in point["path"]:
        trace.append(path)
lines = ipl.Polyline(
    locations=trace,
    color="tomato",
    fill=False
)
m1.add_layer(lines)

# Build client locations layer
base = list(route_dict_flr.values())[0]
start = ipl.Circle(
    location=base["start"],
    radius=30,
    color="DimGray",
    fill_color="Skyblue",
    fill=True,
    fill_opacity=1
)
m1.add_layer(start)

for client in route_dict_flr.values():
    fill_color = colors[int(client["route_id"])]
    circle = ipl.Circle(
        location=client["location"],
        radius=20,
        color="DimGray",
        fill_color=fill_color,
        fill=True,
        fill_opacity=0.85
    )
    m1.add_layer(circle)
    
# Add marker
mark = ipl.Marker(location=base["start"])
m1.add_layer(mark)

m1

Map(center=[52.26842022000208, 6.169152013659209], controls=(ZoomControl(options=['position', 'zoom_in_text', …

### Animation
The the actual sequence of visits can be shown using an animated agent that travels through the plotted route.

In [27]:
step = 0.1
for stage in route_dict_flr.values():
    for point in stage["path"]:
        mark.location = point
        sleep(step)
    visited = ipl.Circle(
        location=stage["location"],
        radius=20,
        color="#00b521",
        fill_color="#7fff96",
        fill=True,
        fill_opacity=0.85
    )
    m1.add_layer(visited)